**Tutorial T'-> tZ analysis**

In [ ]:
# Input parameters
sched_port    = 23356 #Dask port
nmaxpartition = 50
distributed   = False#True#
do_variations = False#True#
hist_folder   = "histos"

if do_variations == True:
    variations = ["nominal", "pu", "jer", "jesTotal"]
else :
    variations = ["nominal"]

in_dataset = ["TprimeToTZ_1800_2022"]#"TT_semilep_2022" #"TT_hadr_2022"

In [ ]:
import ROOT
import os
from samples import *
from variables import *
import json
import numpy as np
import math
from dask.distributed import Client
ROOT.RDF.Experimental.Distributed.open_files_locally = False

os.environ['X509_CERT_DIR'] = "/cvmfs/grid.cern.ch/etc/grid-security/certificates/"
os.environ['X509_USER_PROXY'] = "/tmp/x509up_u0"
print(os.environ.get("X509_USER_PROXY"), os.environ.get("X509_CERT_DIR"))


if distributed:
    nfiles_max = 1000
else:
    nfiles_max = 1  #######

#output histos folder
folder = "./results/"

print("local folder histos: {}".format(folder))
if not os.path.exists(folder):
    os.mkdir(folder)
repohisto = folder+hist_folder+"/"
if not os.path.exists(repohisto):
    os.mkdir(repohisto)

- Import of utils from variables.py
Regions, Variables

- syncro between in_dataset and sample_dict (from sample.py) to syncronize labels and other featurs of the dataset 
- import of samples_dict.json to load files list (path to reach them on tier)


In [ ]:
regions_def = regions # ---> see variables.py
print("Regions to book: ")
for r in regions_def.keys():
    print("  "+r)
    
sample_file = open("dict_samples_2022.json", "rb")
samples = json.load(sample_file)
sample_file.close()

var = vars  # ---> variables.py

print("Variables for histograms :")
print([v._name for v in var])

datasets = []
for in_d in in_dataset:
    if not in_d in sample_dict.keys():
        print("Check the in_dataset string... ", sample_dict.keys())
    else : 
        datasets.append(sample_dict[in_d])
print("Datasets to process : ", [d.label for d in datasets])


chain = {}
ntot_events = {}
for d in datasets:
    if hasattr(d, "components"):
        samples_list = d.components
    else:
        samples_list = [d]
    chain[d.label] = {}
    ntot_events[d.label] = {}
    for s in samples_list:
        if distributed: 
            nfiles = len(samples[d.label][s.label]['strings'])
            for i, string in enumerate(samples[d.label][s.label]['strings']): 
                if distributed:
                    samples[d.label][s.label]['strings'][i] = string.replace("root://cms-xrd-global.cern.ch/", "davs://stwebdav.pi.infn.it:8443/cms/")
                else:
                    samples[d.label][s.label]['strings'][i] = string.replace("root://cms-xrd-global.cern.ch/", "root://stormgf2.pi.infn.it/")
            chain[d.label][s.label] = samples[d.label][s.label]['strings']
        else: 
            nfiles = nfiles_max
            for i, string in enumerate(samples[d.label][s.label]['strings']): 
                if distributed: samples[d.label][s.label]['strings'][i] = string.replace("root://cms-xrd-global.cern.ch/", "davs://stwebdav.pi.infn.it:8443/cms/")
                else: samples[d.label][s.label]['strings'][i] = string.replace("root://cms-xrd-global.cern.ch/", "root://stormgf2.pi.infn.it/")
            chain[d.label][s.label] = samples[d.label][s.label]['strings'][:nfiles]
        if not "Data" in s.label: ntot_events[d.label][s.label] = np.sum(samples[d.label][s.label]['ntot'][:nfiles])
        else: ntot_events[d.label][s.label] = None
        print("Dataset : "+s.label)
        print("# of files to process : ", nfiles)
        if distributed and len(chain[d.label][s.label])>2:
            print("files strings :\n  {}\n  {}\n  ... \n  {}\n  {}".format(chain[d.label][s.label][0], chain[d.label][s.label][1], chain[d.label][s.label][-2], chain[d.label][s.label][-1]))
        else :
            print("files strings :\n  {}".format(chain[d.label][s.label][0]))
        print("# of total events in the files to process (MC only, if Data the number is None) : ", ntot_events[d.label][s.label])

In [ ]:
# initialization of clusters

# upload the proxyfile to the Dask workers to make them able to access data on the grid 

from distributed.diagnostics.plugin import UploadFile
def set_proxy(dask_worker):
    import os
    import shutil
    working_dir = dask_worker.local_directory
    proxy_name = 'x509up_u0'
    os.environ['X509_USER_PROXY'] = working_dir + '/' + proxy_name
    os.environ['X509_CERT_DIR']="/cvmfs/grid.cern.ch/etc/grid-security/certificates/"
    shutil.copyfile(working_dir + '/' + proxy_name, working_dir + '/../../../proxy')    
    os.environ['EXTRA_CLING_ARGS'] = "-O2"
    return os.environ.get("X509_USER_PROXY"), os.environ.get("X509_CERT_DIR")

text_file = open("postselection.h", "r")
data = text_file.read()
def my_initialization_function():
    print(ROOT.gInterpreter.ProcessLine(".O"))
    ROOT.gInterpreter.Declare('{}'.format(data))
    print("end of initialization")

# set up everything properly
if distributed == True:
    RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame
    client = Client(address="tcp://127.0.0.1:"+str(sched_port))
    client.restart()
    # client.register_worker_plugin(UploadFile("/tmp/x509up_u0"))
    client.register_plugin(UploadFile("/tmp/x509up_u0"))
    client.run(set_proxy)
    ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)
else:
    RDataFrame = ROOT.RDataFrame
    my_initialization_function()

In [ ]:
################### utils ###################
def cut_string(cut):
    return cut.replace(" ", "").replace("&&","_").replace(">","_g_").replace(".","_").replace("==","_e_")

################### preselection ###############
def preselection(df):
    df = df.Define("GoodJet_idx", "GetGoodJet(Jet_pt_nominal, Jet_eta, Jet_jetId)")
    df = df.Define("nGoodJet", "nGoodJet(GoodJet_idx)")
    df = df.Define("GoodFatJet_idx", "GetGoodFatJet(FatJet_pt_nominal, FatJet_eta, FatJet_jetId, FatJet_particleNetWithMass_TvsQCD)")
    df = df.Define("nGoodFatJet", "GoodFatJet_idx.size()")
    df = df.Define("minDeltaPhi", "min_DeltaPhi(PuppiMET_T1_phi_nominal, Jet_phi, GoodJet_idx)")
    df = df.Define("nVetoElectron", "nVetoElectron(Electron_pt, Electron_cutBased, Electron_eta)")
    df = df.Define("nVetoMuon", "nVetoMuon(Muon_pt, Muon_eta, Muon_looseId)")
    
    df = df.Filter("nGoodFatJet>0 ", "jet presel")
    df = df.Filter("nVetoElectron+nVetoMuon>0 ", "Lepton Veto")
    df = df.Filter("minDeltaPhi>0.6", "MinDeltaPhi cut")
    
    df = df.Define("nForwardJet", "nForwardJet(Jet_pt_nominal, Jet_jetId, Jet_eta)")    

    return df

############### trigger selection #####################
def trigger_filter(df):
    hlt_met = "(HLT_PFMET120_PFMHT120_IDTight || HLT_PFMETNoMu120_PFMHTNoMu120_IDTight)"
    df_trig = df.Filter(hlt_met, "triggerMET")
    return df_trig

############### top selection ########################
def select_top(df):
    # return indices of the FatJet with particleNet score over the thresholds 
    # df_goodtop      = df.Define("GoodTopMer_idx", "select_TopMer(FatJet_particleNetWithMass_TvsQCD, GoodFatJet_idx)")
        
    df_topselected  = df.Define("Top_idx", "select_bestTop(FatJet_particleNetWithMass_TvsQCD, GoodFatJet_idx)")
    
    df_topvariables = df_topselected.Define("Top_pt", "FatJet_pt_nominal[GoodFatJet_idx[Top_idx]]")\
                        .Define("Top_eta", "FatJet_eta[GoodFatJet_idx[Top_idx]]")\
                        .Define("Top_phi", "FatJet_phi[GoodFatJet_idx[Top_idx]]")\
                        .Define("Top_mass", "FatJet_mass_nominal[GoodFatJet_idx[Top_idx]]")\
                        .Define("Top_score", "FatJet_particleNetWithMass_TvsQCD[GoodFatJet_idx[Top_idx]]")
    return df_topvariables
    
def energetic_variations(df):
    df_sys = df.Vary(["Jet_pt_nominal", "Jet_mass_nominal", "PuppiMET_T1_pt_nominal_vec", "PuppiMET_T1_phi_nominal_vec", "TopMixed_pt_nominal", "TopResolved_pt_nominal", "TopMixed_mass_nominal", "TopResolved_mass_nominal",  "TopMixed_TopScore_nominal", "TopResolved_TopScore_nominal"], "RVec<RVec<RVec<float>>>{{Jet_pt_jerdown, Jet_pt_jerup}, {Jet_mass_jerdown, Jet_mass_jerup}, {PuppiMET_T1_pt_jerdown_vec, PuppiMET_T1_pt_jerup_vec}, {PuppiMET_T1_phi_jerdown_vec, PuppiMET_T1_phi_jerup_vec}, {TopMixed_pt_jerdown, TopMixed_pt_jerup}, {TopResolved_pt_jerdown, TopResolved_pt_jerup}, {TopMixed_mass_jerdown, TopMixed_mass_jerup}, {TopResolved_mass_jerdown, TopResolved_mass_jerup}, {TopMixed_TopScore_jerdown, TopMixed_TopScore_jerup}, {TopResolved_TopScore_jerdown, TopResolved_TopScore_jerup}}", variationTags=["down", "up"], variationName="jer")\
               .Vary(["Jet_pt_nominal", "Jet_mass_nominal", "PuppiMET_T1_pt_nominal_vec", "PuppiMET_T1_phi_nominal_vec", "TopMixed_pt_nominal", "TopResolved_pt_nominal", "TopMixed_mass_nominal", "TopResolved_mass_nominal",  "TopMixed_TopScore_nominal", "TopResolved_TopScore_nominal"], "RVec<RVec<RVec<float>>>{{Jet_pt_jesTotaldown, Jet_pt_jesTotalup}, {Jet_mass_jesTotaldown, Jet_mass_jesTotalup}, {PuppiMET_T1_pt_jesTotaldown_vec, PuppiMET_T1_pt_jesTotalup_vec}, {PuppiMET_T1_phi_jesTotaldown_vec, PuppiMET_T1_phi_jesTotalup_vec}, {TopMixed_pt_jesTotaldown, TopMixed_pt_jesTotalup}, {TopResolved_pt_jesTotaldown, TopResolved_pt_jesTotalup}, {TopMixed_mass_jesTotaldown, TopMixed_mass_jesTotalup}, {TopResolved_mass_jesTotaldown, TopResolved_mass_jesTotalup}, {TopMixed_TopScore_jesTotaldown, TopMixed_TopScore_jesTotalup}, {TopResolved_TopScore_jesTotaldown, TopResolved_TopScore_jesTotalup}}", variationTags=["down", "up"], variationName="jesTotal")
    return df_sys

def SF_variations(df):
    df_sys = df.Vary("puWeight", "RVec<float>{puWeightDown, puWeightUp}", variationTags=["down", "up"], variationName="pu")
    return df_sys

In [ ]:
def bookhisto(df, regions_def, var):
    h_ = {}
    for reg in regions_def.keys():
        h_[reg] = {}
        for v in var:
            if v._MConly and not sampleflag: 
                continue
            else:
                # print(v._name+"_"+reg)
                if regions_def[reg] == "":
                    h_[reg][v._name]= df.Histo1D((v._name+"_"+reg," ;"+v._title+"", v._nbins, v._xmin, v._xmax), v._name, "w_nominal")
                else:
                    h_[reg][v._name]= df.Filter(regions_def[reg]).Histo1D((v._name+"_"+reg," ;"+v._title, v._nbins, v._xmin, v._xmax), v._name, "w_nominal")
    return h_

In [ ]:
def savehisto(d, dict_h, regions_def, var):
    histo = {reg: {v._name: ROOT.TH1D(v._name+"_"+reg," ;"+v._title+"", v._nbins, v._xmin, v._xmax) for v in var} for reg in regions_def.keys()}
    isMC=True
    if "Data" in d.label: isMC = False
    if hasattr(d, "components"):
        s_list = d.components
    else:
        s_list = [d]
    
    for s in s_list:
        outfile = ROOT.TFile.Open(repohisto+s.label+'.root', "RECREATE")

        for n, vari in enumerate(variations):
            for reg in regions_def.keys():
                for v in var:
                    if v._MConly and not isMC:
                        continue
                    else:
                        # da capire come fare il getvalue e dividere le variazioni
                        if isMC:
                            if do_variations:
                                if vari=='nominal':
                                    h1 = dict_h[d.label][s.label][reg][v._name]["nominal"]
                                    h1.SetName(h1.GetName()+"_nominal")
                                    nbins = h1.GetNbinsX()
                                    h1.SetBinContent(1, h1.GetBinContent(0) + h1.GetBinContent(1))
                                    h1.SetBinError(1, math.sqrt(pow(h1.GetBinError(0),2) + pow(h1.GetBinError(1),2)))
                                    h1.SetBinContent(nbins, h1.GetBinContent(nbins) + h1.GetBinContent(nbins+1))
                                    h1.SetBinError(nbins, math.sqrt(pow(h1.GetBinError(nbins),2) + pow(h1.GetBinError(nbins+1),2)))
                                    # if isMC:
                                    #     h1.Scale(s.sigma*10**3/ntot_events[d.label][s.label])
                                    histo_name = h1.GetName()
                                    if "nominal" not in histo_name : h1.SetName(histo_name+"_nominal")
                                    outfile.cd()
                                    h1.Write()
                                else:
                                    for var_type in ['up', 'down']:
                                        h1 = dict_h[d.label][s.label][reg][v._name][vari+":"+var_type]
                                        # h1.SetName(h1.GetName()+"_"+vari+var_type.capitalize())
                                        histo_name = h1.GetName()
                                        if vari+"_"+var_type not in histo_name:
                                            h1.SetName(h1.GetName()+"_"+vari+"_"+var_type)
                                        nbins = h1.GetNbinsX()
                                        h1.SetBinContent(1, h1.GetBinContent(0) + h1.GetBinContent(1))
                                        h1.SetBinError(1, math.sqrt(pow(h1.GetBinError(0),2) + pow(h1.GetBinError(1),2)))
                                        h1.SetBinContent(nbins, h1.GetBinContent(nbins) + h1.GetBinContent(nbins+1))
                                        h1.SetBinError(nbins, math.sqrt(pow(h1.GetBinError(nbins),2) + pow(h1.GetBinError(nbins+1),2)))
                                        # if isMC:
                                        #     h1.Scale(s.sigma*10**3/ntot_events[d.label][s.label])
                                        outfile.cd()
                                        h1.Write()
                            else:
                                histo[reg][v._name] = dict_h[d.label][s.label][reg][v._name].GetValue()      
                                nbins = histo[reg][v._name].GetNbinsX()
                                histo[reg][v._name].SetBinContent(1, histo[reg][v._name].GetBinContent(0) + histo[reg][v._name].GetBinContent(1))
                                histo[reg][v._name].SetBinError(1, math.sqrt(pow(histo[reg][v._name].GetBinError(0),2) + pow(histo[reg][v._name].GetBinError(1),2)))
                                histo[reg][v._name].SetBinContent(nbins, histo[reg][v._name].GetBinContent(nbins) + histo[reg][v._name].GetBinContent(nbins+1))
                                histo[reg][v._name].SetBinError(nbins, math.sqrt(pow(histo[reg][v._name].GetBinError(nbins),2) + pow(histo[reg][v._name].GetBinError(nbins+1),2)))
                                # if isMC:
                                #     histo[reg][v._name].Scale(s.sigma*10**3/ntot_events[d.label][s.label])
                                outfile.cd()
                                histo[reg][v._name].Write()
                        else:
                            histo[reg][v._name] = dict_h[d.label][s.label][reg][v._name].GetValue()
                            nbins = histo[reg][v._name].GetNbinsX()
                            histo[reg][v._name].SetBinContent(1, histo[reg][v._name].GetBinContent(0) + histo[reg][v._name].GetBinContent(1))
                            histo[reg][v._name].SetBinError(1, math.sqrt(pow(histo[reg][v._name].GetBinError(0),2) + pow(histo[reg][v._name].GetBinError(1),2)))
                            histo[reg][v._name].SetBinContent(nbins, histo[reg][v._name].GetBinContent(nbins) + histo[reg][v._name].GetBinContent(nbins+1))
                            histo[reg][v._name].SetBinError(nbins, math.sqrt(pow(histo[reg][v._name].GetBinError(nbins),2) + pow(histo[reg][v._name].GetBinError(nbins+1),2)))
                            # if isMC:
                            #     histo[reg][v._name].Scale(s.sigma*10**3/ntot_events[d.label][s.label])
                            outfile.cd()
                            histo[reg][v._name].Write()
        outfile.Close()

In [ ]:

print("starting loop on datasets: ",[d.label for d in datasets])

h = {}
if do_variations:
    h_varied = {}

for d in datasets:
    s_list = []
    if hasattr(d, "components"):
        s_list = d.components
    else:
        s_list = [d]
    if 'Data' in d.label : sampleflag = 0
    else: sampleflag = 1
    
    h[d.label] = {}
    if do_variations:
        h_varied[d.label]={}
    for s in s_list:
        #-------------------------------------------------------------------------
        #########################  DF initialization #############################
        #-------------------------------------------------------------------------
        
        print("Initializing DataFrame for "+ s.label +" chain len = ", len(chain[d.label][s.label]))
        if len(chain[d.label][s.label])==1: print(chain[d.label][s.label])
        if distributed ==True:
            df = RDataFrame("Events", chain[d.label][s.label], npartitions=nmaxpartition, 
                            daskclient=client, monitor_label = "main" )
        else:
            df = RDataFrame("Events", chain[d.label][s.label])
        df = df.Define("PuppiMET_T1_pt_nominal_vec", "RVec<float>{ (float) PuppiMET_T1_pt_nominal}").Define("PuppiMET_T1_phi_nominal_vec", "RVec<float>{ (float) PuppiMET_T1_phi_nominal}")\
               .Define("PuppiMET_T1_pt_jerdown_vec", "RVec<float>{ (float) PuppiMET_T1_pt_jerdown}").Define("PuppiMET_T1_phi_jerdown_vec", "RVec<float>{ (float) PuppiMET_T1_phi_jerdown}")\
               .Define("PuppiMET_T1_pt_jerup_vec", "RVec<float>{ (float) PuppiMET_T1_pt_jerup}").Define("PuppiMET_T1_phi_jerup_vec", "RVec<float>{ (float) PuppiMET_T1_phi_jerup}")\
               .Define("PuppiMET_T1_pt_jesTotaldown_vec", "RVec<float>{ (float) PuppiMET_T1_pt_jesTotaldown}").Define("PuppiMET_T1_phi_jesTotaldown_vec", "RVec<float>{ (float) PuppiMET_T1_phi_jesTotaldown}")\
               .Define("PuppiMET_T1_pt_jesTotalup_vec", "RVec<float>{ (float) PuppiMET_T1_pt_jesTotalup}").Define("PuppiMET_T1_phi_jesTotalup_vec", "RVec<float>{ (float) PuppiMET_T1_phi_jesTotalup}")
        if do_variations:
            df              = SF_variations(df)
            df              = energetic_variations(df)
        else:
            df              = df
        df_ismc         = df.Define("isMC", "isMC("+str(sampleflag)+")")
        df_year         = df_ismc.Define("year", str(s.year))
        df_hemveto      = df_year.Define("HEMVeto", "hemveto(Jet_eta, Jet_phi, Electron_eta, Electron_phi)")
        df_hemveto      = df_hemveto.Filter("(isMC || (year != 2018) || (HEMVeto || run<319077.))")
        df_hlt          = trigger_filter(df_hemveto)
        
        df_hlt          = df_hlt.Define("w_nominal", "1")
            
        if sampleflag: df_wnom = df_hlt.Redefine('w_nominal', 'w_nominal*puWeight')  
        else: df_wnom          = df_hlt.Redefine('w_nominal', '1')

        df_presel       = preselection(df_wnom)
        df_topsel       = select_top(df_presel)
        df_topsel       = df_topsel.Define("MT_T", "sqrt(2 * Top_pt * PuppiMET_T1_pt_nominal * (1 - cos(Top_phi - PuppiMET_T1_phi_nominal)))")
        
        if len(var) != 0 :
            h[d.label][s.label] = bookhisto(df_topsel, regions_def, var)
        
        if do_variations:
            # h [dataset][label][region][variable]
            print("applying vary")
            h_varied[d.label][s.label]={}
            for reg in regions_def.keys():
                h_varied[d.label][s.label][reg] = {}
                for v in var:
                    if distributed == True:
                        h_varied[d.label][s.label][reg][v._name] = ROOT.RDF.Experimental.Distributed.VariationsFor(h[d.label][s.label][reg][v._name])
                    else:
                        h_varied[d.label][s.label][reg][v._name] = ROOT.RDF.Experimental.VariationsFor(h[d.label][s.label][reg][v._name])


print("All histos booked !")
for d in datasets:
    if len(var):
        if do_variations:
            print(h_varied.keys())
            # print(h_varied[d.label].keys())
            savehisto(d, h_varied, regions_def, var)
        else:
            savehisto(d, h, regions_def, var)
    print(d.label + " histos saved")

In [ ]:
file = ROOT.TFile.Open(repohisto+s.label+".root")
print(file)
for a in file.GetListOfKeys(): print(a)
for reg in regions_def.keys():
    for v in var:
        if do_variations:
            hist = file.Get(v._name+"_"+reg+"_nominal")
        else:
            hist = file.Get(v._name+"_"+reg)
        print(reg, hist.Integral())